# grid
> Set of basic operations on grids

In [ ]:
# | default_exp grid

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *


In [ ]:
# | export
import pandas as pd
from openlocationcode import openlocationcode as olc
from typing import Tuple

Basic OLC validation

In [ ]:
#| export
def get_valid_olc_code(
    olc_code:str # OLC code to validate
    )->str: # Valid OLC code or None
    # Convert a string to a valid OLC code if possible
    if olc.isValid(olc_code):
        return olc_code
    
    if len(olc_code) == 8:
        olc_tmp = f"{olc_code}+"
        return olc_tmp if olc.isValid(olc_tmp) else None
    
    if len(olc_code) > 8:
        olc_tmp = f"{olc_code[:8]}+{olc_code[8:]}"
        return olc_tmp if olc.isValid(olc_tmp) else None
    
    return None

def get_flat_olc_code(
    olc_code:str # OLC code to convert
    )->str: # Valid OLC code without the + 
    
    return olc_code.replace("+", "")

In [ ]:
get_valid_olc_code("9G7V8V8V8VX")

'9G7V8V8V+8VX'

In [ ]:
#| hide
test_eq(get_valid_olc_code("9G7V8V8V+"), "9G7V8V8V+")
test_eq(get_valid_olc_code("9G7V8V8V"), "9G7V8V8V+")
test_eq(get_valid_olc_code("9G7V8V8V8VX"), "9G7V8V8V+8VX")


OLC Coordinates

In [ ]:
# | export
def get_olc_grid_centroid(
    olc_code: str   # OLC code to get centroid for
    ) -> Tuple[float, float]: # Latitude and longitude of centroid
    if olc_code := get_valid_olc_code(olc_code):
        g = olc.decode(olc_code)
        return g.latitudeCenter, g.longitudeCenter
    return None, None

In [ ]:
get_olc_grid_centroid('22VVHQMX+')

(-72.41625, -162.20125)

In [ ]:
#| hide
test_eq(get_olc_grid_centroid('22VVHQMX+'), (-72.41625, -162.20125))

Grid code string operations

In [ ]:
# | export
def olc_to_xy_chars(
    grid: str,
) -> Tuple[str, str]:
    x_chars = "".join([x for i, x in enumerate(grid) if i % 2 == 1])
    y_chars = "".join([x for i, x in enumerate(grid) if i % 2 == 0])

    return x_chars, y_chars


def xy_chars_to_olc(
    x_chars: str,
    y_chars: str,
) -> str:
    return "".join("".join(tup) for tup in zip(y_chars, x_chars))

Grid code arithmetic operations

In [ ]:
# | export
def olc_to_decimal(grid):
    return sum(
        pow(20, i) * olc.CODE_ALPHABET_.index(digit)
        for i, digit in enumerate(grid[::-1])
    )

In [ ]:
test_eq(olc_to_decimal("8FWF6MVX"), 8315077559)

In [ ]:
#| export
def decimal_to_olc(decimal):
    grid = ""
    while decimal > 0:
        grid = olc.CODE_ALPHABET_[decimal % 20] + grid
        decimal //= 20
    return grid

In [ ]:
test_eq(decimal_to_olc(8315077559), "8FWF6MVX")

Grid offset operations

In [ ]:
#| export
def get_offsets_between_grids(
    grid1: str,
    grid2: str,
) -> Tuple[int, int]:
    # Get X and Y axis characters
    x_grid_1, y_grid_1 = olc_to_xy_chars(grid1)
    x_grid_2, y_grid_2 = olc_to_xy_chars(grid2)

    # Get X and Y axis offsets
    x_offset = olc_to_decimal(x_grid_2) - olc_to_decimal(x_grid_1)
    y_offset = olc_to_decimal(y_grid_2) - olc_to_decimal(y_grid_1)

    return x_offset, y_offset

In [ ]:
test_eq(get_offsets_between_grids("9G2243V3", "9G2242XJ"), (-9, 2))
test_eq(get_offsets_between_grids("9G2242XJ", "9G2243V3"), (9, -2))

In [ ]:
#| export
def get_olc_with_offsets(
    grid: str,
    x_offset: int,
    y_offset: int,
) -> str:
    grid = grid.replace("+", "")
    x_grid, y_grid = olc_to_xy_chars(grid)
    x_grid = decimal_to_olc(olc_to_decimal(x_grid) + x_offset)
    y_grid = decimal_to_olc(olc_to_decimal(y_grid) + y_offset)

    return xy_chars_to_olc(x_grid, y_grid)

In [ ]:
test_eq(get_olc_with_offsets("9G2243V3", -9, 2), '9G2242XJ')

In [ ]:
#| hide
# Testing grids at various resolutions
test_eq(get_olc_with_offsets("9G2243V3", -9, 2), '9G2242XJ')
test_eq(get_olc_with_offsets('7JCMHQ', -1, -1), '7JCMGP')
test_eq(get_olc_with_offsets('7JCMHQ9C+2W', 0, 0), '7JCMHQ9C2W')
test_eq(get_olc_with_offsets('84VVHQMX', -1, -1), '84VVHQJW')
test_eq(get_olc_with_offsets('7JCMHQ9C2W', 1, 1), '7JCMHQ9C3X')
test_eq(get_olc_with_offsets('7JCMHQ9C+2W', -1, -1), '7JCMHQ8CXV')

In [ ]:
# | hide
import nbdev; nbdev.nbdev_export()